In [337]:
import numpy as np
import pandas as pd
# for data cleaning 
import re
from gensim.parsing.preprocessing import remove_stopwords # using gensim for stopwords removal 
import nltk
from nltk.stem import PorterStemmer

In [338]:
anime = pd.read_csv('animes.csv')

In [339]:
anime.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",6-Jan-17,1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [340]:
anime.shape

(19311, 12)

In [341]:
anime.head(1)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...


In [342]:
# Cols that are needed for reccomendation system 
# Genre
# id
# synopsis 
# title 

anime = anime[['uid', 'title', 'synopsis', 'genre']]

In [343]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       19311 non-null  int64 
 1   title     19311 non-null  object
 2   synopsis  18336 non-null  object
 3   genre     19311 non-null  object
dtypes: int64(1), object(3)
memory usage: 603.6+ KB


In [344]:
# This is the dataframe where we will build our reccomendation system

anime.head()

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [345]:
# Checking for missing data 
anime.isnull().sum()

uid           0
title         0
synopsis    975
genre         0
dtype: int64

In [346]:
anime.dropna(inplace=True) # dropping rows w/ missing vals 

In [347]:
# Checking for duplicate data
anime.duplicated().sum()

2883

In [348]:
anime = anime.drop_duplicates(keep='first') # keep first duplicate row

In [349]:
anime.iloc[0].genre # The way genre is formatted is ok, we don't have to change anything

"['Comedy', 'Sports', 'Drama', 'School', 'Shounen']"

In [350]:
anime['synopsis'][0]

"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead.  \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but also come up with new attacks that would strengthen them. Moreover, Hinata and Kageyama attempt to devise a more powerful weapon, one that could possibly break the sturdiest of blocks.  \r\n \r\nFacing what may be their last chance at victory before the senior players graduate, the members of Karasuno's volleyball team must learn to settle their differences and train harder than ever if they hope to overcome formidable opponents old and new—including their archrival Aoba Jousai and its world-class 

In [351]:
anime['synopsis'] = anime['synopsis'].apply(lambda x: x.split()) # Turning each word into a list 

In [352]:
anime.head()

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,"[Music, accompanies, the, path, of, the, human...","['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,"[The, Abyss—a, gaping, chasm, stretching, down...","['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"[""In, order, for, something, to, be, obtained,...","['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,"[After, helping, revive, the, legendary, vampi...","['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [353]:
anime.head()

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,"[Music, accompanies, the, path, of, the, human...","['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,"[The, Abyss—a, gaping, chasm, stretching, down...","['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"[""In, order, for, something, to, be, obtained,...","['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,"[After, helping, revive, the, legendary, vampi...","['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [354]:
anime.iloc[0].genre

"['Comedy', 'Sports', 'Drama', 'School', 'Shounen']"

In [355]:
type(anime.iloc[0].synopsis)

genre_list = anime['genre'].tolist() # contains all the genres of each anime 
syn_list = anime['synopsis'].tolist() # contains synopsis of each anime 

tags = [] # will be the concat of genre and synopsis of each anime 

for i in range(len(genre_list)):
    tags.append((" ".join(syn_list[i]) + genre_list[i]).lower()) # making the tag and lowercased it


In [356]:
# Clean the tags
cleaned_tags = []
for tag in tags: 
    tag = re.sub(r"[\[\]]",'',tag) # removing the square brackets 
    tag = remove_stopwords(tag) # removing stopwords
    
    cleaned_tags.append(tag)

In [357]:
cleaned_tags = pd.DataFrame(cleaned_tags)
anime["tags"] = cleaned_tags # appending a new col to the df

In [358]:
anime.head(5)

,uid,title,synopsis,genre,tags
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","following participation inter-high, karasuno h..."
1,23273,Shigatsu wa Kimi no Uso,"[Music, accompanies, the, path, of, the, human...","['Drama', 'Music', 'Romance', 'School', 'Shoun...","music accompanies path human metronome, prodig..."
2,34599,Made in Abyss,"[The, Abyss—a, gaping, chasm, stretching, down...","['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","abyss—a gaping chasm stretching depths earth, ..."
3,5114,Fullmetal Alchemist: Brotherhood,"[""In, order, for, something, to, be, obtained,...","['Action', 'Military', 'Adventure', 'Comedy', ...","""in order obtained, equal value lost."" alchemy..."
4,31758,Kizumonogatari III: Reiketsu-hen,"[After, helping, revive, the, legendary, vampi...","['Action', 'Mystery', 'Supernatural', 'Vampire']",helping revive legendary vampire kiss-shot ace...


In [359]:
new = anime[['uid', 'title', 'tags']].copy() # created a new data frame w/ the cols we will work with 

In [360]:
new['tags']=new['tags'].fillna("") # filling empty tags with empty strings -> handling errors 

In [361]:
new.head(5)

,uid,title,tags
0,28891,Haikyuu!! Second Season,"following participation inter-high, karasuno h..."
1,23273,Shigatsu wa Kimi no Uso,"music accompanies path human metronome, prodig..."
2,34599,Made in Abyss,"abyss—a gaping chasm stretching depths earth, ..."
3,5114,Fullmetal Alchemist: Brotherhood,"""in order obtained, equal value lost."" alchemy..."
4,31758,Kizumonogatari III: Reiketsu-hen,helping revive legendary vampire kiss-shot ace...


In [362]:
new['tags'][1]

"music accompanies path human metronome, prodigious pianist kousei arima. passing mother, saki arima, kousei falls downward spiral, rendering unable hear sound piano. years later, kousei avoids piano, leaving admirers rivals, lives colorless life alongside friends tsubaki sawabe ryouta watari. however, changes meets beautiful violinist, kaori miyazono, stirs world sets journey face music again. based manga series name, shigatsu wa kimi uso approaches story kousei's recovery discovers music playing note perfectly, single melody bring fresh spring air april. written mal rewrite'drama', 'music', 'romance', 'school', 'shounen'"

In [363]:
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [364]:
new['tags'] = new['tags'].apply(stem)

In [365]:
new['tags'][1]

"music accompani path human metronome, prodigi pianist kousei arima. pass mother, saki arima, kousei fall downward spiral, render unabl hear sound piano. year later, kousei avoid piano, leav admir rivals, live colorless life alongsid friend tsubaki sawab ryouta watari. however, chang meet beauti violinist, kaori miyazono, stir world set journey face music again. base manga seri name, shigatsu wa kimi uso approach stori kousei' recoveri discov music play note perfectly, singl melodi bring fresh spring air april. written mal rewrite'drama', 'music', 'romance', 'school', 'shounen'"

In [366]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [367]:
vectors = cv.fit_transform(new['tags'].values.astype('U')).toarray()

In [368]:
vectors 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [369]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [370]:
cv.get_feature_names()

/Users/tasnimmahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '009',
 '01',
 '10',
 '100',
 '10th',
 '11',
 '11th',
 '12',
 '12th',
 '13',
 '13th',
 '14',
 '14th',
 '15',
 '15th',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '18th',
 '19',
 '1945',
 '1984',
 '1985',
 '1989',
 '1995',
 '1998',
 '1999',
 '19th',
 '1st',
 '20',
 '200',
 '2000',
 '2001',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '20th',
 '21',
 '21st',
 '22',
 '22nd',
 '23',
 '24',
 '24th',
 '25',
 '25th',
 '26',
 '27',
 '28',
 '29',
 '2d',
 '2nd',
 '30',
 '300',
 '31',
 '3d',
 '3rd',
 '40th',
 '4th',
 '50',
 '500',
 '5th',
 '60',
 '6th',
 '70',
 '7th',
 '80',
 '8th',
 '90',
 '999',
 '9th',
 'abandon',
 'abduct',
 'abe',
 'abil',
 'abilities',
 'ability',
 'abl',
 'abnorm',
 'aboard',
 'abroad',
 'absent',
 'absolut',
 'absorb',
 'abstract',
 'absurd',
 'abus',
 'academ',
 'academi',
 'academy',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 '